# 3.1 De batch a streaming

En esta sección comparamos dos enfoques sobre el conjunto de datos **Electricity (Elec2)**:

1. **Enfoque batch (Scikit-learn)**  
   - Convertimos el flujo de datos en un `DataFrame` de pandas.  
   - Entrenamos un `GaussianNB` de Scikit-learn sobre un subconjunto de entrenamiento.  
   - Evaluamos sobre un subconjunto de test, respetando el orden temporal (sin barajar).

2. **Enfoque streaming (River)**  
   - Entrenamos un `GaussianNB` de River de forma incremental.  
   - Evaluamos con *evaluación progresiva*: en cada instante predecimos primero y después actualizamos el modelo.

Finalmente comparamos las precisiones obtenidas.


## GaussianNB batch (Scikit-learn)

In [1]:
import pandas as pd
from river import datasets

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# 1) Convertir el flujo Elec2 a DataFrame de pandas
elec_stream = datasets.Elec2()

rows = []
for x, y in elec_stream:
    row = dict(x)
    row['target'] = y
    rows.append(row)

df_elec = pd.DataFrame(rows)
print("Shape del DataFrame:", df_elec.shape)
print(df_elec.head())

# 2) Separar X, y
X = df_elec.drop(columns=['target'])
y = df_elec['target']

# 3) Codificar variables categóricas (por ejemplo 'day')
X_encoded = pd.get_dummies(X)

# 4) Partición train/test respetando el orden (shuffle=False)
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.3, shuffle=False
)

# 5) Entrenar GaussianNB batch de Scikit-learn
gnb_batch = GaussianNB()
gnb_batch.fit(X_train, y_train)

# 6) Evaluar
y_pred_test = gnb_batch.predict(X_test)
acc_batch = accuracy_score(y_test, y_pred_test)

print(f"Precisión GaussianNB (Scikit-learn, batch): {acc_batch:.4f}")


Uncompressing into C:\Users\jorge\river_data\Elec2
Shape del DataFrame: (45312, 9)
   date  day    period  nswprice  nswdemand  vicprice  vicdemand  transfer  \
0   0.0    2  0.000000  0.056443   0.439155  0.003467   0.422915  0.414912   
1   0.0    2  0.021277  0.051699   0.415055  0.003467   0.422915  0.414912   
2   0.0    2  0.042553  0.051489   0.385004  0.003467   0.422915  0.414912   
3   0.0    2  0.063830  0.045485   0.314639  0.003467   0.422915  0.414912   
4   0.0    2  0.085106  0.042482   0.251116  0.003467   0.422915  0.414912   

   target  
0    True  
1    True  
2    True  
3    True  
4   False  
Precisión GaussianNB (Scikit-learn, batch): 0.7539


## GaussianNB streaming (River, evaluación progresiva)

In [ ]:
from river import naive_bayes, metrics, preprocessing, compose, datasets

# 1) Definir modelo GaussianNB de River con preprocesado:
#    - OneHotEncoder para categóricas
model_gnb_stream = compose.Pipeline(
    preprocessing.OneHotEncoder(),
    naive_bayes.GaussianNB()
)

# 2) Métrica de evaluación progresiva
metric_stream = metrics.Accuracy()

progressive_acc_gnb = []

# 3) Recorrer el flujo en orden
dataset_stream = datasets.Elec2()

for i, (x, y) in enumerate(dataset_stream):
    # Predicción antes de aprender
    y_pred = model_gnb_stream.predict_one(x)
    
    # Actualizar métrica
    metric_stream.update(y, y_pred)
    progressive_acc_gnb.append(metric_stream.get())
    
    # Aprender con el ejemplo actual
    model_gnb_stream.learn_one(x, y)

print(f"Precisión final GaussianNB (River, progresiva): {metric_stream.get():.4f}")


## Comparación rápida batch vs streaming

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Curva de precisión progresiva del modelo streaming
plt.figure(figsize=(8,4))
plt.plot(progressive_acc_gnb)
plt.xlabel("Instante (nº de ejemplo en el flujo)")
plt.ylabel("Precisión acumulada")
plt.title("GaussianNB (River) - Evaluación progresiva")
plt.grid(True)
plt.show()

print(f"Precisión GaussianNB batch (Scikit-learn): {acc_batch:.4f}")
print(f"Precisión final GaussianNB streaming (River): {metric_stream.get():.4f}")


: 

# 3.2 Manejo de *concept drift* con ADWIN

En esta parte integramos el detector de cambios **ADWIN** en el flujo de datos:

- Usamos un `GaussianNB` de River como clasificador base.
- Alimentamos a ADWIN con el error (0 si acierta, 1 si falla).
- Cuando ADWIN detecta un cambio relevante:
  - **Estrategia**: reiniciamos el modelo (lo re-inicializamos desde cero).
  - Guardamos el instante en el que se ha producido el *drift* para visualizarlo.

Así obtenemos una versión de GaussianNB que se “resetea” cuando cambia el concepto.


## GaussianNB + ADWIN

In [ ]:
from river import drift

# Modelo base: GaussianNB con OneHotEncoder
def make_gnb_model():
    return compose.Pipeline(
        preprocessing.OneHotEncoder(),
        naive_bayes.GaussianNB()
    )

model_gnb_drift = make_gnb_model()
metric_gnb_drift = metrics.Accuracy()
adwin = drift.ADWIN()

progressive_acc_gnb_drift = []
drift_points = []

dataset_stream = datasets.Elec2()

for i, (x, y) in enumerate(dataset_stream):
    # Predicción antes de aprender
    y_pred = model_gnb_drift.predict_one(x)
    
    # Actualizar métrica
    metric_gnb_drift.update(y, y_pred)
    progressive_acc_gnb_drift.append(metric_gnb_drift.get())
    
    # Actualizar ADWIN con el error (1 si fallo, 0 si acierto)
    error = int(y_pred != y)
    adwin.update(error)
    
    # Si ADWIN detecta *drift*, reiniciamos el modelo
    if adwin.drift_detected:
        print(f"Drift detectado en t = {i}. Reiniciando modelo.")
        drift_points.append(i)
        model_gnb_drift = make_gnb_model()
        adwin = drift.ADWIN()
    
    # Aprendizaje
    model_gnb_drift.learn_one(x, y)

print(f"Precisión final GaussianNB + ADWIN: {metric_gnb_drift.get():.4f}")
print(f"Nº de drifts detectados: {len(drift_points)}")


## Comparar GNB “normal” vs GNB + ADWIN

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(progressive_acc_gnb, label="GNB streaming (sin ADWIN)")
plt.plot(progressive_acc_gnb_drift, label="GNB + ADWIN")
for t in drift_points:
    plt.axvline(t, linestyle="--", alpha=0.3)  # línea vertical en cada drift

plt.xlabel("Instante (nº de ejemplo en el flujo)")
plt.ylabel("Precisión acumulada")
plt.title("Comparación GaussianNB: sin/ con ADWIN")
plt.legend()
plt.grid(True)
plt.show()


# 3.3 Modelos adaptativos: HAT y Adaptive Random Forest

A continuación construimos modelos adaptativos en River:

- Un *pipeline* que combina:
  - `OneHotEncoder` (para manejar atributos categóricos).
  - `StandardScaler` (para normalizar atributos numéricos).
  - Un **Hoeffding Adaptive Tree (HAT)**.

- Un **Adaptive Random Forest (ARF)** que utiliza como árboles base modelos adaptativos.

Para ambos modelos generamos curvas de evaluación progresiva sobre el flujo `Elec2` y las comparamos.


## Función de evaluación progresiva

In [ ]:
from river import tree, ensemble

def progressive_evaluation(model, dataset):
    """Evalúa un modelo River de forma progresiva sobre un dataset en flujo.
    
    Devuelve:
      - lista con la precisión acumulada en cada instante
      - métrica final (Accuracy)
    """
    metric = metrics.Accuracy()
    history = []
    
    for i, (x, y) in enumerate(dataset):
        y_pred = model.predict_one(x)
        metric.update(y, y_pred)
        history.append(metric.get())
        model.learn_one(x, y)
    
    return history, metric


## HAT (Hoeffding Adaptive Tree)

In [ ]:
from river import preprocessing, compose

# Pipeline HAT: OneHotEncoder + StandardScaler + HAT
hat_model = compose.Pipeline(
    preprocessing.OneHotEncoder(),
    preprocessing.StandardScaler(),
    tree.HoeffdingAdaptiveTreeClassifier()
)

dataset_hat = datasets.Elec2()
acc_hat_history, acc_hat_metric = progressive_evaluation(hat_model, dataset_hat)

print(f"Precisión final HAT: {acc_hat_metric.get():.4f}")


## Adaptive Random Forest (ARF)

In [ ]:
# Pipeline ARF: OneHotEncoder + StandardScaler + AdaptiveRandomForest
arf_model = compose.Pipeline(
    preprocessing.OneHotEncoder(),
    preprocessing.StandardScaler(),
    ensemble.AdaptiveRandomForestClassifier(
        n_models=10,      # nº de árboles en el bosque
        seed=42
    )
)

dataset_arf = datasets.Elec2()
acc_arf_history, acc_arf_metric = progressive_evaluation(arf_model, dataset_arf)

print(f"Precisión final ARF: {acc_arf_metric.get():.4f}")


## Comparar curvas HAT vs ARF

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(acc_hat_history, label="HAT")
plt.plot(acc_arf_history, label="Adaptive Random Forest")
plt.xlabel("Instante (nº de ejemplo en el flujo)")
plt.ylabel("Precisión acumulada")
plt.title("Evaluación progresiva: HAT vs Adaptive Random Forest")
plt.legend()
plt.grid(True)
plt.show()


# 3.4 Preguntas de análisis

### 3.4.1 ¿Por qué HAT o ARF tienden a superar a GNB con *concept drift*?

El modelo **Gaussian Naive Bayes (GNB)** asume que:
- Las características son condicionalmente independientes dadas la clase.
- La distribución de cada atributo para cada clase es (normalmente) gaussiana y **estacionaria**.

En un escenario con *concept drift*:

- Las distribuciones de los atributos y/o la relación entre atributos y clase **cambian con el tiempo**.
- GNB mantiene un único conjunto de parámetros globales que se va actualizando con todos los datos vistos.  
  Si el concepto cambia de forma relevante, el modelo arrastra “memoria” del pasado que ya no es representativa.
- Además, la suposición de independencia condicional limita la capacidad para capturar interacciones complejas entre variables.

En cambio, modelos **adaptativos** como **HAT** o **ARF**:

- Se basan en árboles de decisión incrementales que dividen el espacio en regiones, aprendiendo reglas locales.
- Incorporan mecanismos explícitos de detección y adaptación al *drift* (por ejemplo, uso de detectores tipo ADWIN en nodos).
- Pueden reemplazar ramas o subárboles cuando dejan de ser útiles, ajustándose mejor a los nuevos patrones.
- Capturan interacciones no lineales y dependencias entre características sin asumir independencia.

Por todo ello, en presencia de cambios conceptuales, HAT y ARF suelen mantener una precisión más alta y estable a lo largo del tiempo que un GNB simple.

---

### 3.4.2 ¿Por qué ARF suele ser más robusto que HAT?

**Adaptive Random Forest (ARF)** es un **ensamblado** de múltiples árboles adaptativos (por ejemplo, HATs), entrenados con:

- **Bagging online** (poniendo pesos aleatorios a cada ejemplo para cada árbol).
- **Selección aleatoria de subconjuntos de atributos** en cada división (como en Random Forest clásico).

Esto aporta varias ventajas:

1. **Reducción de la varianza**  
   - Un único HAT puede ser muy sensible a cambios en una parte concreta del flujo de datos.  
   - ARF combina muchos árboles y promedia sus predicciones, suavizando el efecto de ruido y drifts locales.

2. **Diversidad de modelos**  
   - Cada árbol ve una versión ligeramente distinta del flujo (por el muestreo tipo bagging) y de las características.  
   - Si el concepto cambia, es probable que algunos árboles se adapten más rápido que otros, manteniendo buen rendimiento global.

3. **Sustitución dinámica de árboles**  
   - ARF suele incorporar detectores de *drift* por árbol.  
   - Cuando un árbol empieza a funcionar mal o se detecta *drift*, puede ser reemplazado por uno nuevo entrenado con los datos recientes.  
   - Este mecanismo de “renovación” hace que el bosque se mantenga actualizado y robusto frente a cambios prolongados.

4. **Mayor estabilidad en presencia de ruido**  
   - El voto mayoritario del conjunto reduce el impacto de ejemplos ruidosos que podrían desestabilizar un único HAT.

Como consecuencia, **ARF suele ser más robusto** que un árbol adaptativo individual (HAT) en escenarios de datos en flujo con *concept drift* y ruido, alcanzando precisiones más altas y con menor variabilidad.
